In [2]:
from urllib.parse import quote, urlparse
import urllib.request
import datetime, os, requests, sys


In [11]:
def download_file(url, filename, timeout=3600):
    if os.path.exists(filename):
        sys.stdout.write('%s already downloaded\n' % filename)
        return True
    else:
        if not os.path.exists(os.path.dirname(filename)):
            os.makedirs(os.path.dirname(filename))
        sys.stdout.write('Downloading %s to %s\n' % (url, filename))

        try:
            response = requests.Session().get(url, timeout=timeout)
            if(response.status_code!=200):
                print('Error response, code = %d, body = %s' % (response.status_code, response.text))
                return False
        except RequestException as e:
            sys.stdout.write("Couldn't read %s because %s" % (url, e))
            return False

        open(filename + '.tmp', "wb").write(response.content)
        os.rename(filename + '.tmp', filename)
        sys.stdout.write('Done, wrote %d bytes to %s\n' % (len(response.content), filename))
        return True

            
class RootArg:
    arg_dict = {
        'v': None,
        't': 0,
        'ps': 0,
        'l': None,
        'bt': None,
        'et': None,
        'startDwell': 0,
        'endDwell': 0,
        'fps': 30
    }
    def __init__(self, v, l, bt, et, **kwargs):
        self.arg_dict['v'] = v
        self.arg_dict['l'] = l
        self.arg_dict['bt'] = bt
        self.arg_dict['et'] = et
        
    def get(self):
        arg_list = []
        for (key,value) in self.arg_dict.items():
            arg_list.append('%s=%s'%(key,value))
        return 'https://headless.earthtime.org/#' + "&".join(arg_list)
    
class ThumbnailUrl:
    arg_dict = {
        'root': None,
        'width': None,
        'height': None,
        'format': 'png',
        'fps': 30,
        'tileFormat': 'mp4',
        'startDwell': 0,
        'endDwell': 0,
        'fromScreenshot': True
    }
    
    def __init__(self, root, width, height):
        self.arg_dict['root'] = root
        self.arg_dict['width'] = width
        self.arg_dict['height'] = height
        

    def get(self):
        arg_list = []
        for (key,value) in self.arg_dict.items():
            arg_list.append('%s=%s'%(key,value))
        return 'https://hal40-thumbnailworker.createlab.org/thumbnail?' + "&".join(arg_list)
    

def export_landsat_images_from_earthtime(sharelink, odir=None, w='5120', h='2880'):
    years = range(1984,2023)
    dates = ['%s0401' % x for x in years]
    u = urlparse(sharelink)
    v = None
    l = None
    for fragment in u.fragment.split('&'):
        key,val = fragment.split('=')
        if key == 'v':
            v = val
        if key == 'l':
            l = val
    if v is None or l is None:
        print('Malformed sharelink')
        return False
    
    if odir is None:
        now = datetime.datetime.now()
        timestamp = int(now.timestamp())
        odir = '%s_%s' % (l.replace(',','_'), timestamp)

    os.makedirs(odir, exist_ok=True)
    thumbnails = []
    for date in dates:
        bt = et = date
        root_arg = RootArg(v,l,bt,et)
        thumbnail_url = ThumbnailUrl(quote(root_arg.get(), safe=''), w, h)
        fname = str(date[0:4]) + '.png'
        thumbnails.append((thumbnail_url.get(), os.path.join(odir,fname)))
    for t in thumbnails:
        download_file(t[0], t[1])
    return True

def export_images_from_earthtime(sharelink, start_date=1984, end_date=2022, odir=None, w='5120', h='2880'):
    years = range(start_date,end_date+1)
    dates = []
    for y in years:
        for m in range(1,13):
            dates.append('%s%s01' % (str(y), str(m).zfill(2)))
    u = urlparse(sharelink)
    v = None
    l = None
    for fragment in u.fragment.split('&'):
        key,val = fragment.split('=')
        if key == 'v':
            v = val
        if key == 'l':
            l = val
    if v is None or l is None:
        print('Malformed sharelink')
        return False
    
    if odir is None:
        now = datetime.datetime.now()
        timestamp = int(now.timestamp())
        odir = '%s_%s' % (l.replace(',','_'), timestamp)

    os.makedirs(odir, exist_ok=True)
    thumbnails = []
    for date in dates:
        bt = et = date
        root_arg = RootArg(v,l,bt,et)
        thumbnail_url = ThumbnailUrl(quote(root_arg.get(), safe=''), w, h)
        fname = str(date) + '.png'
        thumbnails.append((thumbnail_url.get(), os.path.join(odir,fname)))
    for t in thumbnails:
        print(t[0])
        download_file(t[0], t[1])
    return True

def export_images_from_earthtime_annual(sharelink, start_date=1984, end_date=2022, odir=None, w='5120', h='2880'):
    years = range(start_date,end_date+1)
    dates = []
    for y in years:
        for m in range(1,2):
            dates.append('%s%s01' % (str(y), str(m).zfill(2)))
    u = urlparse(sharelink)
    v = None
    l = None
    for fragment in u.fragment.split('&'):
        key,val = fragment.split('=')
        if key == 'v':
            v = val
        if key == 'l':
            l = val
    if v is None or l is None:
        print('Malformed sharelink')
        return False
    
    if odir is None:
        now = datetime.datetime.now()
        timestamp = int(now.timestamp())
        odir = '%s_%s' % (l.replace(',','_'), timestamp)

    os.makedirs(odir, exist_ok=True)
    thumbnails = []
    for date in dates:
        bt = et = date
        root_arg = RootArg(v,l,bt,et)
        thumbnail_url = ThumbnailUrl(quote(root_arg.get(), safe=''), w, h)
        fname = str(date) + '.png'
        thumbnails.append((thumbnail_url.get(), os.path.join(odir,fname)))
    for t in thumbnails:
        print(t[0])
        download_file(t[0], t[1])
    return True

    
    

In [ ]:
#export_landsat_images_from_earthtime('https://earthtime.org/explore#v=36.15028,-114.54353,9.608,latLng&t=3.2&ps=50&l=blsat&bt=19840101&et=20221231')

In [ ]:
export_landsat_images_from_earthtime('https://earthtime.org/explore#v=36.08499,-114.73683,10.109,latLng&t=0&ps=50&l=blsat&bt=19840101&et=20221231')

In [9]:
sharelink = 'https://earthtime.org/explore#v=36.08499,-114.73683,10.109,latLng&t=0&ps=50&l=blsat&bt=19840101&et=20221231'
start_year = 2000
end_year = 2015
out_dir = None
export_images_from_earthtime(sharelink, start_year, end_year, out_dir)

https://hal40-thumbnailworker.createlab.org/thumbnail?root=https%3A%2F%2Fheadless.earthtime.org%2F%23v%3D32.46328%2C-97.35808%2C2.056%2ClatLng%26t%3D0%26ps%3D0%26l%3Dbdrk%2C50_percent_probability_RCP85_tas_seasonal_JJA%26bt%3D20000101%26et%3D20000101%26startDwell%3D0%26endDwell%3D0%26fps%3D30&width=5120&height=2880&format=png&fps=30&tileFormat=mp4&startDwell=0&endDwell=0&fromScreenshot=True
Done, wrote 1443557 bytes to bdrk_50_percent_probability_RCP85_tas_seasonal_JJA_1714762340/20000101.png
https://hal40-thumbnailworker.createlab.org/thumbnail?root=https%3A%2F%2Fheadless.earthtime.org%2F%23v%3D32.46328%2C-97.35808%2C2.056%2ClatLng%26t%3D0%26ps%3D0%26l%3Dbdrk%2C50_percent_probability_RCP85_tas_seasonal_JJA%26bt%3D20000201%26et%3D20000201%26startDwell%3D0%26endDwell%3D0%26fps%3D30&width=5120&height=2880&format=png&fps=30&tileFormat=mp4&startDwell=0&endDwell=0&fromScreenshot=True
Done, wrote 1443557 bytes to bdrk_50_percent_probability_RCP85_tas_seasonal_JJA_1714762340/20000201.png
http

NameError: name 'RequestException' is not defined

In [13]:
sharelink = 'https://earthtime.org/explore#v=36.08499,-114.73683,10.109,latLng&t=0&ps=50&l=blsat&bt=19840101&et=20221231'
start_year = 2050
end_year = 2060
out_dir = None
export_images_from_earthtime_annual(sharelink, start_year, end_year, out_dir)

https://hal40-thumbnailworker.createlab.org/thumbnail?root=https%3A%2F%2Fheadless.earthtime.org%2F%23v%3D36.08499%2C-114.73683%2C10.109%2ClatLng%26t%3D0%26ps%3D0%26l%3Dblsat%26bt%3D20500101%26et%3D20500101%26startDwell%3D0%26endDwell%3D0%26fps%3D30&width=5120&height=2880&format=png&fps=30&tileFormat=mp4&startDwell=0&endDwell=0&fromScreenshot=True


NameError: name 'RequestException' is not defined